In [1]:
from graphviz import Digraph
from copy import deepcopy

In [2]:
def move2(src,path):
    global DFA
    
    # 根据初始状态点和转移函数获取结束状态
    for i in range(len(DFA)):
        if(DFA[i][0]==src and DFA[i][1]==path):
            return DFA[i][2]
    return ' '

In [3]:
def get_source_state(des_state_set,path):
    global allstatus

    #allStatus=set(allstatus)
    src_set=set()
    
    for state in allstatus:
        if move2(state,path) in des_state_set:
            src_set.update(state)
            #合并字典或集合的内容
            #如果在合并过程中有相同的键或元素，它将覆盖旧的键值对或元素
    
    return src_set

In [4]:
def hopcroft():
    global endset
    global allstatus
    global paths

    #将终止状态集合和全部状态集转化为集合，方便之后做减法
    #endSet=set(endset)
    #allStatus=set(allstatus)
    #paths=set(sigma)
    
    #初始化终止状态和非终止状态
    terminated_states=endset
    not_terminated_states=allstatus-terminated_states
    
    P = [terminated_states, not_terminated_states]
    # P 是状态分组的列表
    W = [terminated_states, not_terminated_states]
    # W 是待处理的状态的列表
    
    while W:
        #select and remove S from W
        S=W[0]
        W.remove(S)
        #选择一个A（未处理的状态集合的首部）
        #在W中移除（即将开始处理A）
        
        for path in paths:
            # 对每一个输入字符，查找对能转换到当前状态的状态集合la ← δ^-1(S, a)
            la=get_source_state(S,path)
            
            # 存储分隔后的新状态集合
            new_split_state=[]
            
            #for all R in P such that R ∩ la ≠ ∅ and R ∉ la do
            for R in P:
                condition1=R&la
                condition2=R-la
                
                if len(condition1) and len(condition2):
                    
                    #partition R into R1 and R2: R1 ← R ∩ la and R2 ← R − R1
                    #不能在这里直接对P进行操作，P在之后还要遍历呢
                    new_split_state.append(condition1)
                    new_split_state.append(condition2)
                
                    #if R ∈ W then
                    if R in W:
                        
                        #replace R in W with R1 and R2
                        W.remove(R)
                        W.append(condition1)
                        W.append(condition2)
                        
                    #else
                    else:
                        
                        #if |R1| ≤ |R2| then
                        if len(condition1)<=len(condition2):
                        
                            #add R1 to W
                            W.append(condition1)
                            
                        #else
                        else:
                            
                            #add R2 to W
                            W.append(condition2)

                else:
                    new_split_state.append(R)
                    
            #replace R in P with R1 and R2
            P=deepcopy(new_split_state)
    return P           
                            

In [5]:
def getdata_dfa():
    global paths
    global DFA
    global allstatus
    global dot
    global endset
    
    #打开文件并读取数据
    f=open('data.txt')
    content=f.readlines()
    src_num=len(content)
    f.close()
    
    for i in range(src_num):
        temp=content[i].split()
        #temp是一个列表，其中有每一行的所有元素
        
        DFA.append(temp)
        paths.append(temp[1])
        allstatus.append(temp[0])
        allstatus.append(temp[2])
        
        if temp[0]==DFA[0][0]:
            if temp[0] not in endset:
                dot.node(name=temp[0],color='blue')
            else:
                dot.node(name=temp[0],color='blue',shape='doublecircle')
        elif temp[0] in endset:
            dot.node(name=temp[0],color='red',shape='doublecircle')
        else:
            dot.node(name=temp[0],color='black')
            
        dot.edge(temp[0], temp[2], temp[1])
    
    #使用集合去重
    paths=set(paths)
    allstatus=set(allstatus)

In [6]:
def getdata_end():
    global endset
    
    f=open('endstate.txt')
    content=f.readlines()
    line_num=len(content)
    f.close()
    
    for i in range(line_num):
        endset.append(content[i][0])

    endset=set(endset)
    

In [7]:
def printprint(printsth):
    for i in range(len(printsth)):
        print(printsth[i])

In [8]:
paths=[]
endset=[]
allstatus=[]

DFA=[]

In [9]:
dot = Digraph("DFA")

In [10]:
getdata_end()
getdata_dfa()

In [11]:
print("DFA:")
printprint(DFA)

DFA:
['S', 'a', 'A']
['S', 'b', 'B']
['A', 'a', 'C']
['A', 'b', 'B']
['B', 'a', 'A']
['B', 'b', 'D']
['C', 'a', 'C']
['C', 'b', 'E']
['D', 'a', 'F']
['D', 'b', 'D']
['E', 'a', 'F']
['E', 'b', 'D']
['F', 'a', 'C']
['F', 'b', 'E']


In [12]:
dot.render('DFA', view=True)

'DFA.pdf'

In [13]:
minDFA = hopcroft()

In [14]:
print("minDFA:")
printprint(minDFA)

minDFA:
{'E', 'C', 'D', 'F'}
{'A'}
{'B'}
{'S'}


In [15]:
result_DFA=[]

In [16]:
def get_index_in_minDFA(state):
    #找到当前状态state在minDFA中的索引值
    for i in range(len(minDFA)):
        if state in minDFA[i]:
            return i
    return ' '

In [17]:
def get_new_path():
    global minDFA
    global paths
    global result_DFA

    Paths=list(paths)
    MinDFA=list(minDFA)
    #将集合转换成列表，方便访问下标

    for i in range(len(MinDFA)):
        #对于每一个状态，找到其可以到达的新状态
        states=list(MinDFA[i])

        #用于存储新的DFA的每一种状态经过各path后到的新状态
        dst_states = [0] * len(Paths)

        for j in range(len(states)):
            #对于其中的每一个状态

            for path in range(len(Paths)):
                #对于每一种输入

                if dst_states[path]==0 and move2(states[j],Paths[path]) !=' ':
                    ptr=get_index_in_minDFA(move2(states[j],Paths[path]))
                    #ptr是当前状态当前输入下的新状态在minDFA中的索引值

                    dst_states[path]=min(MinDFA[ptr])
                    #此时要找到ptr索引值对应的minDFA的那一个状态数组的最小值（重新命名）

        for j in range(len(Paths)):
            if dst_states[j]!=0:
                temp=[]
                temp.append(min(minDFA[i]))
                temp.append(Paths[j])
                temp.append(dst_states[j])

                result_DFA.append(temp)


In [18]:
get_new_path()
print("result_DFA:")
printprint(result_DFA)

result_DFA:
['C', 'a', 'C']
['C', 'b', 'C']
['A', 'a', 'C']
['A', 'b', 'B']
['B', 'a', 'A']
['B', 'b', 'C']
['S', 'a', 'A']
['S', 'b', 'B']


In [19]:
dot2 = Digraph("newDFA")

In [20]:
for i in range(len(result_DFA)):
    if result_DFA[i][0] in endset:
        dot2.node(name=result_DFA[i][0],color="red",shape='doublecircle')
    else:
        dot2.node(name=result_DFA[i][0],color="black")

    dot2.edge(result_DFA[i][0],result_DFA[i][2],result_DFA[i][1])

In [21]:
dot2.render('minDFA', view=True)

'minDFA.pdf'